# Getting started with tomsup

Tomsup, Theory of Mind Simulation using Python, is a Python Package for Agent Based simulations. It includes 1) a framework for running agent based simulations using 2 by 2 payoffmatrices and most notably 2) an implementation of game theory of mind in a agent based framework following the implementation of [Devaine, et al. (2017)](http://dx.plos.org/10.1371/journal.pcbi.1005833).

This tutorial will simply introduce the framework of tomsup, for an introduction to the theory of mind (ToM) agent, see *introduction_to_tom.ipynb*.

It is also possible for create your own agents for an introduction on this see *creating_an_agent.ipynb*. 

Lastly we have also created an brief introduction to each of the simpler agents for this see *introduction_to_basic_agents.ipynb*.


In [1]:
#assuming you are in the github folder change the path - not relevant if tomsup is installed via. pip
import os
print(os.getcwd())
os.chdir("..") # go out of the tutorials folder
print(os.getcwd())

/mnt/xinyuanyan/IGSN/tom_in_network/tomsup-master/tutorials
/mnt/xinyuanyan/IGSN/tom_in_network/tomsup-master


In [2]:
import tomsup as ts

To get an overview of possible agent you can use the following function. Give you a brief description of their strategy as well as a reference for further reading.

In [3]:
ts.valid_agents()
ts.TOM(level = 1, volatility= 1,  b_temp= -1, bias = 0, dilution = None, save_history = True)


---
## 1) Creating an agent
First we will set up a Random bias (RB) agent. This agent simply choses randomly with a given bias.
There is two ways to setup an agent, either using the agent class (e.g. RB) or using the ```create_agents()``` function. We will start by calling the agent class RB directy. For a whole list of valid agent use the ```ts.valid_agents()```.

In [4]:
jung = ts.RB(bias = 0.7, save_history = True) #calling the agent subclass RB - for more on save_history see '3) inspecting Agent and AgentGroup'

# Let's examine the jung
print(f"jung is an class of type: {type(jung)}")
if isinstance(jung, ts.Agent):
    print(f"but jung is also an instance of the parent class ts.Agent")

# let us have Jung make a choice 
choice = jung.compete()

print(f"jung chose {choice} and his probability for choosing 1 was {jung.get_bias()}.")

jung is an class of type: <class 'tomsup.agent.RB'>
but jung is also an instance of the parent class ts.Agent
jung chose 1 and his probability for choosing 1 was 0.7.


As previously mentioned you can also create agents using the `create_agent` function. Here we will create skinner as a Q-learning agent, which is a simple reinforcement learning agent, see Watkinns (1992) for more.

In [5]:
skinner = ts.create_agents(agents = "QL", start_params = {'save_history': True}) # create a reinforcement learning agent

Since skinner is a reinforcement learning agent his compete function requires him to know which game he is playing, so he can choose based on payoff. He also needs to know his opponents move during their last turn, so that he can update his belief about his opponent choices.

Let us have jung and skinner play the the matching pennies game. We can fetch the game from the function `PayoffMatrix`,

In [6]:
penny = ts.PayoffMatrix(name = "penny_competitive") # fetch the competitive matching pennies game.

#print the payoff matrix
print(penny)

#fetch the underlying numpy matrix
print(penny.get_matrix())

<Class PayoffMatrix, Name = penny_competitive> 
The payoff matrix of agent 0
       |  Choice agent 1
       |    |  0 |  1 |
       | ------------ |
Choice |  0 |  1 | -1 |
agent 0|  1 | -1 |  1 |
 
The payoff matrix of agent 1
       |  Choice agent 1
       |    |  0 |  1 |
       | ------------ |
Choice |  0 | -1 |  1 |
agent 0|  1 |  1 | -1 |
 
[[[ 1 -1]
  [-1  1]]

 [[-1  1]
  [ 1 -1]]]


Let us try to have skinner and jung compete in the matching pennies game:

In [7]:
jung_a = jung.compete() # a for action
skinner_a = skinner.compete(p_matrix = penny, agent = 1, op_choice = None) #Note that op_choice can be unspecified (or None) in the first round

jung_p = penny.payoff(action_agent0 = jung_a, action_agent1 = skinner_a, agent = 0)
skinner_p = penny.payoff(action_agent0 = jung_a, action_agent1 = skinner_a, agent = 1)

print(f"jung chose {jung_a} and skinner chose {skinner_a}, which results in a payoff for jung of {jung_p} and skinner of {skinner_p}.")
# Note that you might get different results simply by chance

jung chose 1 and skinner chose 0, which results in a payoff for jung of -1 and skinner of 1.


---
## 2) Running a tournament
In the above case we saw how to have two agents compete for a single round. It is however rare that we only need 1 round and while the above functionality can be wrapped within a for we have made a `compete()` function for convenience. In this section we will also examine the class `AgentGroup`, which allows you to run tournaments with multiple agents.

Let us start with having the two agent compete for 30 rounds in the matching pennies game:

In [8]:
results = ts.compete(jung, skinner, p_matrix=penny, n_rounds = 30, save_history=True)
print(type(results))

jung_sum = results['payoff_agent0'].sum()
skinner_sum = results['payoff_agent1'].sum()

print(f"jung seemed to get a total of {jung_sum} points, while skinner got a total of {skinner_sum}.")

results.head() #inspect the first 5 rows of the df

<class 'tomsup.plot.ResultsDf'>
jung seemed to get a total of -8 points, while skinner got a total of 8.


/mnt/xinyuanyan/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


,round,choice_agent0,choice_agent1,payoff_agent0,payoff_agent1,history_agent0,history_agent1
0,0,1,1,1,-1,{'choice': 1},"{'choice': 1, 'expected_value0': 0.5, 'expecte..."
1,1,0,0,1,-1,{'choice': 0},"{'choice': 0.0, 'expected_value0': 0.5, 'expec..."
2,2,0,0,1,-1,{'choice': 0},"{'choice': 0.0, 'expected_value0': 0.5, 'expec..."
3,3,1,0,-1,1,{'choice': 1},"{'choice': 0.0, 'expected_value0': 0.5, 'expec..."
4,4,1,0,-1,1,{'choice': 1},"{'choice': 0.0, 'expected_value0': 0.5, 'expec..."


In [9]:
# xinyuanyan
# Dec,2020
# Key words: ToM, penny-competitive game, complex network, computational modeling


# assuming you are in the github folder change the path - not relevant if tomsup is installed via. pip
import math
import os
import numpy as np
import matplotlib.pyplot as plt

import tomsup as ts
print(os.getcwd())
os.chdir("..") # go out of the tutorials folder
print(os.getcwd())
# creat ToM agents
ts.valid_agents()
# ts.TOM()
# Create a 2-TOM agent with volatility -2, a low temperature -4, a bias of 0.5 and a dilution of 0.4.


# We can extract the given parameter values of the agents
# print(sir_TOM0.get_parameters())
# Get the competitive penny game payoff matrix
penny = ts.PayoffMatrix("penny_competitive")


def exponential_func(x, aval):  # define exponential function here
    y = math.pow(aval, x)
    return y





/mnt/xinyuanyan/IGSN/tom_in_network/tomsup-master
/mnt/xinyuanyan/IGSN/tom_in_network


In [10]:
# now let the agent in each cell activate and interact!

totalsteps = 1000#000  # 1000 steps with newborns
#totalround = 20  # in 1 step, how many rounds will the two agents interact?
iB = 5  # intial benefits for every agent in 0 step 0 round
total_round = 10
tomagents = ['0-TOM', '1-TOM', '2-TOM', '3-TOM']
# setting the potential energy-cost ratio
energy_a = [1.1,  1.3,  1.5,  1.7,  1.9, 2.0, 2.1]  # y = a^x (a-value)

all_type_results = np.zeros(shape=(len(energy_a), int(totalsteps), len(tomagents)))


In [12]:
#read the interacting pairs from excel
import pandas as pd
whichhub = ['3tom_hub','2tom_hub','1tom_hub','0tom_hub']

#print(os.getcwd())
os.chdir("/mnt/xinyuanyan/IGSN/tom_in_network/tomsup-master/tutorials/") # go out of the tutorials folder

for hub in [1]:#range(len(whichhub)):
    
    interact_matrix = pd.read_excel('IG_scalefree_prepare_network.xlsx',sheet_name=whichhub[hub],usecols = 'C:F')
    


fB = iB*np.ones(shape=(len(interact_matrix), 2))  # final benefits, we need to update it ! but with initial 1 units in each grid cell
interact_matrix
#print(fB)
#let us get tommat
tommat = np.zeros(shape = (len(interact_matrix),2))
for ktom in range(len(interact_matrix)):

    tommat[ktom][0] = interact_matrix.ktom0[ktom]
    tommat[ktom][1] = interact_matrix.ktom1[ktom]
    
    
#print(tommat)

We see that the output of the compete function if a pandas dataframe. It is possible to change this to a list by specifying `return_val = "list"`, but having it as a dataframe allow for convenient operators attributes such as mean() and sum().

The above case a the simplest possible version of the `compete()` function. You can also specify number of simulations, whether the agent should be reset after each simulation (this is recommended) and whether it should print what simulation it is running (`silent`).




**Note** that by adding simulations the dataframe now also have a column called 'n_sim', for which simulations in which the results belongs.

### AgentGroup
Now as promised, let us take a look at tournaments with multiple agents. We will start of by creating a group of agents using a list of the desired agents as well as a list of their starting parameters. If you are in doubt how to specify these you can always the starting parameters from an existing agent using `jung.get_start_params()`.

In [ ]:
#now we have the interact_matrix for each possible round
for EC in range(len(energy_a)):

    tomagent_sv_ratio = np.zeros(shape=(int(totalsteps), len(tomagents)))

    for step in range(totalsteps):
        
        for ktom in range(len(interact_matrix)):

        # now interact and collect data
            final_out_payoff = []
            final_out_central_payoff = []



            all_agents = [interact_matrix.agent0[ktom],
                                      interact_matrix.agent1[ktom]]        
    

            agents = all_agents#['RB', 'QL', 'WSLS'] # create a list of agents
            start_params = [{}, {}]#[{'bias': 0.7}, {'learning_rate': 0.5}, {}] # create a list of their starting parameters (an empty dictionary {} simply assumes defaults)

            group = ts.create_agents(agents, start_params) # create a group of agents
            print(group)
            print("\n----\n") # to space out the outputs

            group.set_env(env = 'round_robin') # round_robin e.g. each agent will play against all other agents

            # make them compete
            group.compete(p_matrix = penny, n_rounds = 20, n_sim = 4)
            results = group.get_results()
            results.head() #examine the first 5 rows in results
            df = group.head(total_round)

            # get the mean payoff of each agent
            final_out_perstep = df.describe()
            final_out_central_payoff.append(
                            final_out_perstep.loc['mean'].payoff_agent0)  
            
            final_out_payoff.append(
                            final_out_perstep.loc['mean'].payoff_agent1)

                    # now let us update the fB for each agent mentioned above
            fB[ktom][0] =fB[ktom][0]+np.mean(final_out_central_payoff)#agent0 
            fB[ktom][1] = fB[ktom][1]+np.mean(final_out_payoff)#agent1
        
        ######################现在在fB的基础上，要减去各个agent消耗的能量######################
        current_energy = fB
        # get the ratio for all grid cells
        tomlevel = tommat
        energy_cost_ratio = np.zeros(shape=(len(fB),2))
        for ktom in range(len(fB)):
            energy_cost_ratio[ktom][0] = exponential_func(tommat[ktom][0], energy_a[EC])
            energy_cost_ratio[ktom][1] = exponential_func(tommat[ktom][1], energy_a[EC])
        
        # save the updated energy
        this_step_enery = current_energy - current_energy * energy_cost_ratio / 100
        fB = this_step_enery  # updated fB val

        # to see who die? if die, he will be replaced by the agent who has maxium value
        r, c = np.where(this_step_enery == np.max(this_step_enery))
        print(r, c)
        max_agent = tommat[r[0]][c[0]]
        tommat_new = np.zeros(shape=(10, 10))

        for ktom in range(len(fB)):

            if this_step_enery[ktom][0] < 0:
                tommat[ktom][0] = max_agent
            if this_step_enery[ktom][1] < 0:
                tommat[ktom][1] = max_agent

        # tommat = tommat_new#update tommat for next step
        # collect the survived ratio
        tomagent_sv_ratio[step][0] = np.sum(tommat == 0) / 100
        tomagent_sv_ratio[step][1] = np.sum(tommat == 1) / 100
        tomagent_sv_ratio[step][2] = np.sum(tommat == 2) / 100
        tomagent_sv_ratio[step][3] = np.sum(tommat == 3) / 100
        #
        print('finished computating ratio!')

        # plot the results for each a (energy_cost ratio a-val)
        all_type_results[EC][step][0] = np.sum(tommat == 0) / 100
        all_type_results[EC][step][1] = np.sum(tommat == 1) / 100
        all_type_results[EC][step][2] = np.sum(tommat == 2) / 100
        all_type_results[EC][step][3] = np.sum(tommat == 3) / 100
        
        print('finished computating ratio2!')


<Class AgentGroup, envinment = None> 

2-ToM	 | 	{}
0-ToM	 | 	{}

----

Currently the pair, ('2-ToM', '0-ToM'), is competing for 4                         simulations, each containg 20 rounds.
	Running simulation 1 out of 4
	Running simulation 2 out of 4
	Running simulation 3 out of 4
	Running simulation 4 out of 4
Simulation complete
<Class AgentGroup, envinment = None> 

2-ToM	 | 	{}
1-ToM	 | 	{}

----

Currently the pair, ('2-ToM', '1-ToM'), is competing for 4                         simulations, each containg 20 rounds.
	Running simulation 1 out of 4
	Running simulation 2 out of 4
	Running simulation 3 out of 4
	Running simulation 4 out of 4
Simulation complete
<Class AgentGroup, envinment = None> 

2-ToM	 | 	{}
1-ToM	 | 	{}

----

Currently the pair, ('2-ToM', '1-ToM'), is competing for 4                         simulations, each containg 20 rounds.
	Running simulation 1 out of 4
	Running simulation 2 out of 4
	Running simulation 3 out of 4
	Running simulation 4 out of 4
Simulatio

As you can see once the group is created and environment it is easy to have the agent compete with one another.

(for more possible environment, see `help(group.set_env)`)

In [ ]:
all_type_results